In [18]:
from MUA import *
from Vis import rho_scatter_3d
from phy.plot import View
from hdbscan import HDBSCAN
import phy
from phy.gui import GUI
import seaborn as sns
import vispy.scene
from vispy.scene import visuals
from vispy.visuals.transforms import STTransform
%gui qt

In [19]:
mua = MUA(filename='S:/pcie.bin', nCh=32, fs=25000, numbytes=4)
spk = mua.tospk()

In [13]:
fet = spk.tofet('pca')

In [14]:
from ipywidgets import interact

In [17]:
@interact(ch=(0,31,1))
def spike_sort(ch=0):
    min_cluster_size = 20
#     min_samples = 3
    leaf_size = 30
    hdbcluster = HDBSCAN(min_cluster_size=min_cluster_size, 
                     leaf_size=leaf_size,
                     gen_min_span_tree=True, 
#                      min_samples = min_samples,
                     algorithm='best')
    clu = hdbcluster.fit_predict(fet[ch])
    print 'get clusters', np.unique(clu)

    gui = GUI(position=(0, 0), size=(600, 400), name='GUI')

    canvas = vispy.scene.SceneCanvas(keys='interactive', show=False, fullscreen=False)
    scatterview = canvas.central_widget.add_view()
    canvas.unfreeze()
    gui.add_view(canvas, name=str(ch), position='left')
    scatter = rho_scatter_3d(fet[ch][:,:3],clu=clu)
    scatterview.add(scatter)
    scatterview.camera = 'turntable'  # or try 'arcball'
    # add a colored 3D axis for orientation
    axis = visuals.XYZAxis(parent=scatterview.scene)
    
    nclu = len(np.unique(clu))
    view = View(layout='grid',  shape=(3, nclu))
    gui.add_view(view, name=str(ch), position='right')
    palette = sns.color_palette()

    view.clear()
    for chNo in range(3):
        for clu_id in np.unique(clu):
            color = palette[clu_id] if clu_id>=0 else np.array([1,1,1])
            view[chNo,clu_id+1].plot(y=spk[ch][clu==clu_id,:,chNo].squeeze(),
                                                  color = np.hstack((color, 0.2)),
                                                  data_bounds=(-2, -1800, 2, 800))
    view.build()
    gui.show()

get clusters [-1]
